## Week 2 Day 2

Our first Agentic Framework project!!

Prepare yourself for something ridiculously easy.

We're going to build a simple Agent system for generating cold sales outreach emails:
1. Agent workflow
2. Use of tools to call functions
3. Agent collaboration via Tools and Handoffs

## Before we start - some setup:


Please visit Sendgrid at: https://sendgrid.com/

(Sendgrid is a Twilio company for sending emails.)

If SendGrid gives you problems, see the alternative implementation using "Resend Email" in community_contributions/2_lab2_with_resend_email

Please set up an account - it's free! (at least, for me, right now).

Once you've created an account, click on:

Settings (left sidebar) >> API Keys >> Create API Key (button on top right)

Copy the key to the clipboard, then add a new line to your .env file:

`SENDGRID_API_KEY=xxxx`

And also, within SendGrid, go to:

Settings (left sidebar) >> Sender Authentication >> "Verify a Single Sender"  
and verify that your own email address is a real email address, so that SendGrid can send emails for you.


In [ ]:
from dotenv import load_dotenv
from agents import Agent, Runner, trace, function_tool
from openai.types.responses import ResponseTextDeltaEvent
from typing import Dict
import sendgrid
import os
from sendgrid.helpers.mail import Mail, Email, To, Content
import asyncio



In [ ]:
load_dotenv(override=True)

In [ ]:
# Let's just check emails are working for you

def send_test_email():
    sg = sendgrid.SendGridAPIClient(api_key=os.environ.get('SENDGRID_API_KEY'))
    from_email = Email(os.environ.get("EMAIL"))  # Change to your verified sender
    to_email = To(os.environ.get("EMAIL"))  # Change to your recipient
    content = Content("text/plain", "This is an important test email")
    mail = Mail(from_email, to_email, "Test email", content).get()
    response = sg.client.mail.send.post(request_body=mail)
    print(response.status_code)

send_test_email()

### Did you receive the test email

If you get a 202, then you're good to go!

#### Certificate error

If you get an error SSL: CERTIFICATE_VERIFY_FAILED then students Chris S and Oleksandr K have suggestions:  
First run this: `!uv pip install --upgrade certifi`  
Next, run this:
```python
import certifi
import os
os.environ['SSL_CERT_FILE'] = certifi.where()
```

#### Other errors or no email

If there are other problems, you'll need to check your API key and your verified sender email address in the SendGrid dashboard

Or use the alternative implementation using "Resend Email" in community_contributions/2_lab2_with_resend_email

(Or - you could always replace the email sending code below with a Pushover call, or something to simply write to a flat file)

## Step 1: Agent workflow

In [ ]:
instructions1 = "You are a sales agent working for ComplAI, \
a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. \
You write professional, serious cold emails."

instructions2 = "You are a humorous, engaging sales agent working for ComplAI, \
a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. \
You write witty, engaging cold emails that are likely to get a response."

instructions3 = "You are a busy sales agent working for ComplAI, \
a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. \
You write concise, to the point cold emails."

In [ ]:
sales_agent1 = Agent(
        name="Professional Sales Agent",
        instructions=instructions1,
        model="gpt-4o-mini"
)

sales_agent2 = Agent(
        name="Engaging Sales Agent",
        instructions=instructions2,
        model="gpt-4o-mini"
)

sales_agent3 = Agent(
        name="Busy Sales Agent",
        instructions=instructions3,
        model="gpt-4o-mini"
)

In [ ]:

result = Runner.run_streamed(sales_agent1, input="Write a cold sales email")
async for event in result.stream_events():
    if event.type == "raw_response_event" and isinstance(event.data, ResponseTextDeltaEvent):
        print(event.data.delta, end="", flush=True)

In [ ]:
message = "Write a cold sales email"

with trace("Parallel cold emails"):
    results = await asyncio.gather(
        Runner.run(sales_agent1, message),
        Runner.run(sales_agent2, message),
        Runner.run(sales_agent3, message),
    )

outputs = [result.final_output for result in results]

for output in outputs:
    print(output + "\n\n")


In [ ]:
sales_picker = Agent(
    name="sales_picker",
    instructions="You pick the best cold sales email from the given options. \
Imagine you are a customer and pick the one you are most likely to respond to. \
Do not give an explanation; reply with the selected email only.",
    model="gpt-4o-mini"
)

In [ ]:
message = "Write a cold sales email"

with trace("Selection from sales people"):
    results = await asyncio.gather(
        Runner.run(sales_agent1, message),
        Runner.run(sales_agent2, message),
        Runner.run(sales_agent3, message),
    )
    outputs = [result.final_output for result in results]

    emails = "Cold sales emails:\n\n" + "\n\nEmail:\n\n".join(outputs)

    best = await Runner.run(sales_picker, emails)

    print(f"Best sales email:\n{best.final_output}")


Now go and check out the trace:

https://platform.openai.com/traces

## Part 2: use of tools

Now we will add a tool to the mix.

Remember all that json boilerplate and the `handle_tool_calls()` function with the if logic..

In [ ]:
sales_agent1 = Agent(
        name="Professional Sales Agent",
        instructions=instructions1,
        model="gpt-4o-mini",
)

sales_agent2 = Agent(
        name="Engaging Sales Agent",
        instructions=instructions2,
        model="gpt-4o-mini",
)

sales_agent3 = Agent(
        name="Busy Sales Agent",
        instructions=instructions3,
        model="gpt-4o-mini",
)

In [ ]:
sales_agent1

## Steps 2 and 3: Tools and Agent interactions

Remember all that boilerplate json?

Simply wrap your function with the decorator `@function_tool`

In [ ]:
@function_tool
def send_email(body: str):
    """ Send out an email with the given body to all sales prospects """
    sg = sendgrid.SendGridAPIClient(api_key=os.environ.get('SENDGRID_API_KEY'))
    from_email = Email(os.environ.get("EMAIL"))  # Change to your verified sender
    to_email = To(os.environ.get("EMAIL"))  # Change to your recipient
    content = Content("text/plain", body)
    mail = Mail(from_email, to_email, "Sales email", content).get()
    sg.client.mail.send.post(request_body=mail)
    return {"status": "success"}

### This has automatically been converted into a tool, with the boilerplate json created

In [ ]:
# Let's look at it
send_email

### And you can also convert an Agent into a tool

In [ ]:
tool1 = sales_agent1.as_tool(tool_name="sales_agent1", tool_description="Write a cold sales email")
tool1

### So now we can gather all the tools together:

A tool for each of our 3 email-writing agents

And a tool for our function to send emails

In [ ]:
description = "Write a cold sales email"

tool1 = sales_agent1.as_tool(tool_name="sales_agent1", tool_description=description)
tool2 = sales_agent2.as_tool(tool_name="sales_agent2", tool_description=description)
tool3 = sales_agent3.as_tool(tool_name="sales_agent3", tool_description=description)

tools = [tool1, tool2, tool3, send_email]

tools

## And now it's time for our Sales Manager - our planning agent

In [ ]:
# Improved instructions thanks to student Guillermo F.

instructions = """
You are a Sales Manager at ComplAI. Your goal is to find the single best cold sales email using the sales_agent tools.
 
Follow these steps carefully:
1. Generate Drafts: Use all three sales_agent tools to generate three different email drafts. Do not proceed until all three drafts are ready.
 
2. Evaluate and Select: Review the drafts and choose the single best email using your judgment of which one is most effective.
 
3. Use the send_email tool to send the best email (and only the best email) to the user.
 
Crucial Rules:
- You must use the sales agent tools to generate the drafts — do not write them yourself.
- You must send ONE email using the send_email tool — never more than one.
"""


sales_manager = Agent(name="Sales Manager", instructions=instructions, tools=tools, model="gpt-4o-mini")

message = "Send a cold sales email addressed to 'Dear CEO'"

with trace("Sales manager"):
    result = await Runner.run(sales_manager, message)

<table style="margin: 0; text-align: left; width:100%">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/stop.png" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#ff7800;">Wait - you didn't get an email??</h2>
            <span style="color:#ff7800;">With much thanks to student Chris S. for describing his issue and fixes. 
            If you don't receive an email after running the prior cell, here are some things to check: <br/>
            First, check your Spam folder! Several students have missed that the emails arrived in Spam!<br/>Second, print(result) and see if you are receiving errors about SSL. 
            If you're receiving SSL errors, then please check out theses <a href="https://chatgpt.com/share/680620ec-3b30-8012-8c26-ca86693d0e3d">networking tips</a> and see the note in the next cell. Also look at the trace in OpenAI, and investigate on the SendGrid website, to hunt for clues. Let me know if I can help!
            </span>
        </td>
    </tr>
</table>

### And one more suggestion to send emails from student Oleksandr on Windows 11:

If you are getting certificate SSL errors, then:  
Run this in a terminal: `uv pip install --upgrade certifi`

Then run this code:
```python
import certifi
import os
os.environ['SSL_CERT_FILE'] = certifi.where()
```

Thank you Oleksandr!

## Remember to check the trace

https://platform.openai.com/traces

And then check your email!!


### Handoffs represent a way an agent can delegate to an agent, passing control to it

Handoffs and Agents-as-tools are similar:

In both cases, an Agent can collaborate with another Agent

With tools, control passes back

With handoffs, control passes across



In [ ]:

subject_instructions = "You can write a subject for a cold sales email. \
You are given a message and you need to write a subject for an email that is likely to get a response."

html_instructions = "You can convert a text email body to an HTML email body. \
You are given a text email body which might have some markdown \
and you need to convert it to an HTML email body with simple, clear, compelling layout and design."

subject_writer = Agent(name="Email subject writer", instructions=subject_instructions, model="gpt-4o-mini")
subject_tool = subject_writer.as_tool(tool_name="subject_writer", tool_description="Write a subject for a cold sales email")

html_converter = Agent(name="HTML email body converter", instructions=html_instructions, model="gpt-4o-mini")
html_tool = html_converter.as_tool(tool_name="html_converter",tool_description="Convert a text email body to an HTML email body")


In [ ]:
@function_tool
def send_html_email(subject: str, html_body: str) -> Dict[str, str]:
    """ Send out an email with the given subject and HTML body to all sales prospects """
    sg = sendgrid.SendGridAPIClient(api_key=os.environ.get('SENDGRID_API_KEY'))
    from_email = Email(os.environ.get("EMAIL"))  # Change to your verified sender
    to_email = To(os.environ.get("EMAIL"))  # Change to your recipient
    content = Content("text/html", html_body)
    mail = Mail(from_email, to_email, subject, content).get()
    sg.client.mail.send.post(request_body=mail)
    return {"status": "success"}

In [ ]:
tools = [subject_tool, html_tool, send_html_email]

In [ ]:
tools

In [ ]:
instructions ="You are an email formatter and sender. You receive the body of an email to be sent. \
You first use the subject_writer tool to write a subject for the email, then use the html_converter tool to convert the body to HTML. \
Finally, you use the send_html_email tool to send the email with the subject and HTML body."


emailer_agent = Agent(
    name="Email Manager",
    instructions=instructions,
    tools=tools,
    model="gpt-4o-mini",
    handoff_description="Convert an email to HTML and send it")


### Now we have 3 tools and 1 handoff

In [ ]:
tools = [tool1, tool2, tool3]
handoffs = [emailer_agent]
print(tools)
print(handoffs)

In [ ]:
# Improved instructions thanks to student Guillermo F.

sales_manager_instructions = """
You are a Sales Manager at ComplAI. Your goal is to find the single best cold sales email using the sales_agent tools.
 
Follow these steps carefully:
1. Generate Drafts: Use all three sales_agent tools to generate three different email drafts. Do not proceed until all three drafts are ready.
 
2. Evaluate and Select: Review the drafts and choose the single best email using your judgment of which one is most effective.
You can use the tools multiple times if you're not satisfied with the results from the first try.
 
3. Handoff for Sending: Pass ONLY the winning email draft to the 'Email Manager' agent. The Email Manager will take care of formatting and sending.
 
Crucial Rules:
- You must use the sales agent tools to generate the drafts — do not write them yourself.
- You must hand off exactly ONE email to the Email Manager — never more than one.
"""


sales_manager = Agent(
    name="Sales Manager",
    instructions=sales_manager_instructions,
    tools=tools,
    handoffs=handoffs,
    model="gpt-4o-mini")

message = "Send out a cold sales email addressed to Dear CEO from Alice"

with trace("Automated SDR"):
    result = await Runner.run(sales_manager, message)

### Remember to check the trace

https://platform.openai.com/traces

And then check your email!!

<table style="margin: 0; text-align: left; width:100%">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/exercise.png" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#ff7800;">Exercise</h2>
            <span style="color:#ff7800;">Can you identify the Agentic design patterns that were used here?<br/>
            What is the 1 line that changed this from being an Agentic "workflow" to "agent" under Anthropic's definition?<br/>
            Try adding in more tools and Agents! You could have tools that handle the mail merge to send to a list.<br/><br/>
            HARD CHALLENGE: research how you can have SendGrid call a Callback webhook when a user replies to an email,
            Then have the SDR respond to keep the conversation going! This may require some "vibe coding" 😂
            </span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left; width:100%">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/business.png" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#00bfff;">Commercial implications</h2>
            <span style="color:#00bfff;">This is immediately applicable to Sales Automation; but more generally this could be applied to  end-to-end automation of any business process through conversations and tools. Think of ways you could apply an Agent solution
            like this in your day job.
            </span>
        </td>
    </tr>
</table>

## Extra note:

Google has released their Agent Development Kit (ADK). It's not yet got the traction of the other frameworks on this course, but it's getting some attention. It's interesting to note that it looks quite similar to OpenAI Agents SDK. To give you a preview, here's a peak at sample code from ADK:

```
root_agent = Agent(
    name="weather_time_agent",
    model="gemini-2.0-flash",
    description="Agent to answer questions about the time and weather in a city.",
    instruction="You are a helpful agent who can answer user questions about the time and weather in a city.",
    tools=[get_weather, get_current_time]
)
```

Well, that looks familiar!

And a student has contributed a customer care agent in community_contributions that uses ADK.

In [ ]:
# This requires additional free API tokens to be put into your .env file
# IPINFO_TOKEN - https://ipinfo.io/
# SERP_API - https://serpapi.com/
# both have free tier tokens
# Also requires EMAIL=your@email.com in your .env file

# This script will get your location
# Find local news reports in your location
# Write short summaries of those articles
# Have another AI select which summary is intersting
# Compose an email with the summary and an image of the event
# BROKE - the image can't load due to security reasons.

# imports from before:
from dotenv import load_dotenv
from agents import Agent, Runner, trace, function_tool
from openai.types.responses import ResponseTextDeltaEvent
from typing import Dict
import sendgrid
import os
from sendgrid.helpers.mail import Mail, Email, To, Content
import asyncio

# load your .env file
load_dotenv(override=True)

# imports needed for this:
import ipinfo
import os
import requests
import serpapi as sapi # pip install serpapi
import base64
from bs4 import BeautifulSoup


class GetLocalNews:
    def __init__(self) -> None:
        # ----------------------------------------------
        # Replace with your API token from ipinfo.io
        # ----------------------------------------------
        access_token = os.environ.get("IPINFO_TOKEN")

        # Initialize the IPinfo handler
        self.handler = ipinfo.getHandler(access_token)

        # The IP address you want to look up
        ip_response = requests.get('https://api.ipify.org?format=json')
        ip_response.raise_for_status()
        self.ip_address = ip_response.json()['ip']

    def get_location(self):
        try:
            return self.handler.getDetails(self.ip_address).all
        except Exception as e:
            return {}
    
    def get_local_news(self):
        location = self.get_location()
        params = {
            "api_key": os.environ.get("SERPAPI_TOKEN"),
            "engine": "google_news",
            "hl": "en",
            "gl": location.get("country").lower(),
            "q": f"{location.get('city')}, {location.get('region')} Local News"
        }

        news = sapi.search(params)
        return news.get("news_results")


@function_tool
def image_tag() -> str:
    return f"<img class='local_news_image' />"

@function_tool
def get_local_news():
    gln = GetLocalNews()
    return gln.get_local_news()

pick_artical_instructions = f"""
Use your tool to fetch local news. Your goal is to pick one that sounds interesting, ideally within the last 3 months.

The JSON output from that will have this structure:

[
    {{
        'position': number, 
        'title': title,
        'source':
            {{
                'name': name,
                'icon': url
            }},
        'link': url,
        'thumbnail': url,
        'thumbnail_small': url,
        'date': date
    }}
]

Return JSON in this format as your output:

{{
    'title': title,
    'link': url,
    'thumbnail': url
    'date': date
}}

Only return the JSON as described as your output. Do not change any of the data that you found from fetching the local news.
Your only goal is to select an artical and output the right data to the proper portions of the JSON.
"""

summary_instructions1 = f"""
You work for a company called "Brought2ULocally". Your job is to write a short summary about a news artical whose URL will be
supplied to you. Use your tool to retrieve JSON in this form:

{{
    'title': title,
    'link': url,
    'thumbnail': url
    'date': date
}}

Summary instructions:

 - The title of the artical is stored in "title" - include that at the top of summary as the title.
 - The date is in "date". Be sure to put it underneath the title.
 - The URL you will need to use to read the artical is in the "link" portion of the JSON.
 - Sign off with the company name.

 Return JSON output like so:

 {{
    'summary': the summary you made
    'thumbnail: the thumbnail url you received from the JSON returned from your tool - do not change this url
 }}

Do not change the thumbnail url. It needs to be the same as the url from the input JSON.

Return only the JSON described as the output.
"""

summary_instructions2 = f"""
You work for a company called "Brought2ULocally". Your job is to write a short summary about a news artical whose URL will be
supplied to you. Try to make a bad pun as the conclusion of your summary. Use your tool to retrieve JSON in this form:

{{
    'title': title,
    'link': url,
    'thumbnail': url
    'date': date
}}

Summary instructions:

 - The title of the artical is stored in "title" - include that at the top of summary as the title.
 - The date is in "date". Be sure to put it underneath the title.
 - The URL you will need to use to read the artical is in the "link" portion of the JSON.
 - Sign off with the company name.

 Return JSON output like so:

 {{
    'summary': the summary you made
    'thumbnail: the thumbnail url you received from the JSON returned from your tool - do not change this url
 }}

Do not change the thumbnail url. It needs to be the same as the url from the input JSON.

Return only the JSON described as the output.
"""

summary_instructions3 = f"""
You work for a company called "Brought2ULocally". Your job is to write a short summary about a news artical whose URL will be
supplied to you. Try to make it feel like the world is ending because of this event as the conclusion of your summary. Use your
tool to retrieve JSON in this form:

{{
    'title': title,
    'link': url,
    'thumbnail': url
    'date': date
}}

Summary instructions:

 - The title of the artical is stored in "title" - include that at the top of summary as the title.
 - The date is in "date". Be sure to put it underneath the title.
 - The URL you will need to use to read the artical is in the "link" portion of the JSON.
 - Sign off with the company name.

 Return JSON output like so:

 {{
    'summary': the summary you made
    'thumbnail: the thumbnail url you received from the JSON returned from your tool - do not change this url
 }}

Do not change the thumbnail url. It needs to be the same as the url from the input JSON.

Return only the JSON described as the output.
"""

summary_picker_instructions = f"""
You pick your favorite summary given to you at "Brought2ULocally". Your goal is to find the your favorite email using the summary agent tools.
 
Follow these steps carefully:
1. Generate Summaries: Use all three summary agent tools to generate three different summaries stored in their JSON response 'summary' section.
Do not proceed until all three are ready.
 
2. Evaluate and Select: Review the drafts from the 'summary' section of the JSON response and choose the single best summary using your judgment
of which one is most effective. You can use each tool twice, if you're not satisfied with the results from the first try.

3. Handoff for Sending: Pass the JSON response you received to the Email Manager. The Email Manager will take care of formatting and sending.

Important Notes:
 - The tools you use will return summaries in this JSON format:

{{
'summary': the summary
'thumbnail: a thumbnail url - do not change this url
}}

So be sure to read the summary portion and pick the one to your liking.

 - Do not change the thumbnail url.

Crucial Rules:
- You must use the summary agent tools to generate the JSON responses with the summaries — do not write them yourself.
- You must hand off exactly ONE JSON response to the Email Manager — never more than one.
- Your output is to match the following JSON:

 {{
    'summary': the summary you selected
    'thumbnail: the thumbnail url from the in the JSON object you used to find the summary you liked - do not change this url
 }}
- Do not change the thumbnail url - this comes directly from the JSON returned from the summary tools.
- Only respond in JSON when you handoff the response to the Email Manager. You should be passing along the same JSON that you
received from your tool to the Email Manager.
"""

subject_instructions = """
You can write a subject for a short summary.
You are given a message and you need to write a subject for an email that is likely to catch someone's attention.
"""

html_instructions = """
You are given input JSON in this format:

 {{
    'summary': HTML body
    'thumbnail: a thumbnail URL - do not change this url
 }}

- You can convert the summary into an HTML email body.
- You are given a text email body which might have some markdown
and you need to convert it to an HTML email body with simple, clear, compelling layout and design.
- You also have access to a URL that is a thumbnail that would be a nice touch to include in the email. You have a tool that you can use
to do this image_to_data_uri, which will return an <img> tag with the src attribute filled out. You must pass the URL as a string
to it to use it. The URL must come from the input JSON's "thumbnail" section. It must be the exact URL nothing else. Do not change the URL.
Do not change the output from the tool. Assume that it is always right.
"""

subject_writer = Agent(name="Email subject writer", instructions=subject_instructions, model="gpt-4o-mini")
subject_tool = subject_writer.as_tool(tool_name="subject_writer", tool_description="Write a subject for a short summary email")

html_converter = Agent(name="HTML email body converter", tools=[image_tag], instructions=html_instructions, model="gpt-4o-mini")
html_tool = html_converter.as_tool(tool_name="html_converter", tool_description="Convert a text email body to an HTML email body")

def image_to_data_uri(image_url: str) -> str:
    try:
        response = requests.get(image_url)
        response.raise_for_status()  # Raise an exception for bad status codes

        image_data = response.content
        encoded_image = base64.b64encode(image_data).decode('utf-8')

        # Determine image format for the data URI (e.g., 'jpeg', 'png')
        # This is a basic attempt; more robust methods might inspect image headers.
        content_type = response.headers.get('Content-Type', 'application/octet-stream')
        if 'image/' in content_type:
            image_format = content_type.split('/')[-1]
        else:
            image_format = 'png' # Default to png if content type is not clear

        CHUNK_SIZE = 76
        base64_chunks = [encoded_image[i:i + CHUNK_SIZE] for i in range(0, len(encoded_image), CHUNK_SIZE)]
        formatted_base64 = '\n'.join(base64_chunks)
        
        return f"data:image/{image_format};base64,{formatted_base64}"

    except requests.exceptions.RequestException as e:
        print(f"Error downloading image: {e}")
        return ""
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
        return ""

def refine_html(body):
    soup = BeautifulSoup(body, 'html.parser')

    # Correct way to find an <img> tag with a specific class
    # Use the keyword argument 'class_' because 'class' is a reserved word in Python
    local_news_image = soup.find('img', class_='local_news_image')

    # Check if the element was actually found before trying to access its attributes
    if local_news_image:
        original_src = local_news_image.get('src')
        if original_src:
            # Modify the src attribute
            local_news_image['src'] = image_to_data_uri(original_src)

    # Return the HTML
    return str(soup)

# No matter how hard I try, I can't get it to include an image of from the news report
# due to different security issues with sending images in email.
@function_tool
def send_html_email(subject: str, html_body: str) -> Dict[str, str]:
    """ Send out an email with the given subject and HTML body to all sales prospects """
    sg = sendgrid.SendGridAPIClient(api_key=os.environ.get('SENDGRID_API_KEY'))
    from_email = Email(os.environ.get("EMAIL"))  # Change to your verified sender
    to_email = To(os.environ.get("EMAIL"))  # Change to your recipient
    # content = Content("text/html", refine_html(html_body))
    content = Content("text/html", html_body)
    mail = Mail(from_email, to_email, subject, content).get()
    sg.client.mail.send.post(request_body=mail)
    return {"status": "success"}

instructions ="""
You are an email formatter and sender. You receive JSON in this format:

 {{
    'summary': the email body
    'thumbnail: a url to a thumbnail - do not change this url
 }}

There are some things you need to know:
- The JSON has 2 portions to it:
    - summary - The email body
    - thumbnail - A string that is a URL; do not change this

Instructions:
1) You first use the subject_writer tool to write a subject for the email, using the summary.
2) You will send all and ONLY all of the JSON you received as input to the html_converter tool to convert the summary to HTML
and use the thumbnail in the email. Do not change the <img> tag that gets generated from the image_to_data_uri tool.
3) Finally, you use the send_html_email tool to send the email with the subject and HTML body that you received.

You will need to pass on the input JSON to the html_converter for formatting.
"""

emailer_agent = Agent(
    name="Email Manager",
    instructions=instructions,
    tools=[subject_tool, html_tool, send_html_email],
    model="gpt-4o-mini",
    handoff_description="Convert an email to HTML and send it")

interesting_news_agent = Agent(
    name="Interesting News Agent",
    instructions=pick_artical_instructions,
    tools=[get_local_news],
    model="gpt-4o-mini"
).as_tool(tool_name="interesting_artical", tool_description="Picks an interesting news artical")

news_summarizer1 = Agent(
        name="Local News Summarizing Agent 1",
        instructions=summary_instructions1,
        tools=[interesting_news_agent],
        model="gpt-4o-mini"
).as_tool(tool_name="news_summarizer1", tool_description="Writes a summary")
news_summarizer2 = Agent(
        name="Local News Summarizing Agent 2",
        instructions=summary_instructions2,
        tools=[interesting_news_agent],
        model="gpt-4o-mini"
).as_tool(tool_name="news_summarizer2", tool_description="Writes a summary")
news_summarizer3 = Agent(
        name="Local News Summarizing Agent 3",
        instructions=summary_instructions3,
        tools=[interesting_news_agent],
        model="gpt-4o-mini"
).as_tool(tool_name="news_summarizer3", tool_description="Writes a summary")

summary_picker = Agent(
    name="Summary Picker Agent",
    instructions=summary_picker_instructions,
    tools=[news_summarizer1, news_summarizer2, news_summarizer3],
    handoffs=[emailer_agent], # reuse the emailer agent
    model="gpt-4o-mini")


message = "Use your tools to find your favorite summary and then email it."

with trace("Automated Local News Artical Summary"):
    result = await Runner.run(summary_picker, message)